# Running queries on the source data

## Install required packages

Before executing the cells below, in a terminal session install the following required R packages:

`conda create -n omop-source r-tidyverse r-data.table r-dbi  r-rpostgres r-irkernel -y`

### Connect to the SQL database

In [ ]:
library(tidyverse)
library(data.table)
library(DBI)
library(RPostgres)

DBNAME <- #<Add here relevant password>
HOST <- #<Add here relevant password>
PORT <- #<Add here relevant port>
PASSWORD <-  #<Add here relevant password>
USER <- 'jupyter_notebook'

connection <- DBI::dbConnect(
    RPostgres::Postgres(),
    dbname = $DBNAME,
    host = $HOST,
    port = $PORT,
    password = $PASSWORD,
    user = $USER,
    )

### List all tables in the source_data schema

In [ ]:
sql <- "
SELECT table_schema, table_name 
FROM information_schema.tables 
WHERE table_schema = 'source_data'
"

dbGetQuery(connection, sql)

### Query 1: Normalised Rare Disease Terms Cohort 

In [ ]:
sql <- "
SELECT DISTINCT participant_id, normalised_specific_disease
FROM rare_diseases_participant_disease
WHERE normalised_specific_disease IN (
    'Familial pulmonary fibrosis','Familial primary spontaneous pneumothorax',
    'Familial and multiple pulmonary arteriovenous malformations','Hereditary haemorrhagic telangiectasia'
)
"

sql_to_run <- str_replace_all(sql, "rare_diseases_participant_disease", "gel_rare_participant_disease_100k")

dbGetQuery(connection, sql_to_run)

### Query 2: Normalised Rare Disease Terms and Participant Phenotypic Sex Cohort 

In [ ]:
sql <- "
SELECT DISTINCT rd.participant_id, rd.normalised_specific_disease, par.participant_phenotypic_sex
FROM rare_diseases_participant_disease as rd
LEFT JOIN participant AS par
    ON rd.participant_id = par.participant_id
WHERE rd.normalised_specific_disease IN (
    'Familial pulmonary fibrosis','Familial primary spontaneous pneumothorax',
    'Familial and multiple pulmonary arteriovenous malformations','Hereditary haemorrhagic telangiectasia')
AND par.participant_phenotypic_sex IN ('Female')
"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))rare_diseases_participant_disease", "gel_rare_participant_disease_100k") %>%
    str_replace_all("(?<=(FROM )|(JOIN ))participant ", "gel_participant_100k ") 

dbGetQuery(connection, sql_to_run)

### Query 3: Normalised Rare Disease Terms, Participant Phenotypic Sex Cohort, Age of Onset, and Predicted Ancestry

In [ ]:
sql <- "
SELECT DISTINCT rd.participant_id, rd.normalised_specific_disease, par.participant_phenotypic_sex, rd.normalised_age_of_onset,
                agg.pred_european_ancestries
FROM rare_diseases_participant_disease as rd
LEFT JOIN participant AS par
    ON rd.participant_id = par.participant_id
LEFT JOIN aggregate_gvcf_sample_stats AS agg
    ON rd.participant_id = agg.participant_id
WHERE rd.normalised_specific_disease IN (
    'Familial pulmonary fibrosis','Familial primary spontaneous pneumothorax',
    'Familial and multiple pulmonary arteriovenous malformations','Hereditary haemorrhagic telangiectasia'
)
AND par.participant_phenotypic_sex IN ('Female')
AND rd.normalised_age_of_onset >= 50
AND agg.pred_european_ancestries >= 0.95
"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))rare_diseases_participant_disease", "gel_rare_participant_disease_100k") %>%
    str_replace_all("(?<=(FROM )|(JOIN ))participant ", "gel_participant_100k ") %>%
    str_replace_all("(?<=(FROM )|(JOIN ))aggregate_gvcf_sample_stats ", "gel_rare_disease_and_germline_genomic_variant_call_format_sampl ")

dbGetQuery(connection, sql_to_run)

### Query 4: HPO terms cohort

In [ ]:
sql <- "
SELECT DISTINCT participant_id
FROM rare_diseases_participant_phenotype
WHERE normalised_hpo_id IN ('HP:0002206','HP:0006530','HP:0002094')
AND hpo_present IN ('Yes')
"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))rare_diseases_participant_phenotype", "gel_rare_participant_phenotype_100k")

dbGetQuery(connection, sql_to_run) 

### Query 5: ICD10 terms cohort

In [ ]:
sql <- "
SELECT DISTINCT participant_id
FROM hes_apc
WHERE diag_01 LIKE 'J841'
OR diag_02 LIKE 'J841'
OR diag_03 LIKE 'J841'
"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))hes_apc", "nhs_d_hospital_episodes_statistics_admitted_patient_care_100k")

dbGetQuery(connection, sql_to_run)

### Query 6: ICD10 terms cohort - Part 2

In [ ]:
sql <- "
SELECT DISTINCT apc.participant_id
FROM hes_apc AS apc
LEFT JOIN hes_op AS op
    ON apc.participant_id = op.participant_id
WHERE apc.diag_01 IN ('J841')
OR apc.diag_02 IN ('J841')
OR apc.diag_03 IN ('J841')
OR op.diag_01 IN ('J841')
OR op.diag_02 IN ('J841')
OR op.diag_03 IN ('J841')
"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))hes_apc", "nhs_d_hospital_episodes_statistics_admitted_patient_care_100k") %>%
    str_replace_all("(?<=(FROM )|(JOIN ))hes_op ", "nhs_d_hospital_episodes_statistics_outpatient_100k ") 

dbGetQuery(connection, sql_to_run)

### Query 7: Cancer type and status

In [ ]:
sql <- "
SELECT DISTINCT ca.participant_id
FROM cancer_analysis AS ca
LEFT JOIN av_tumour AS av
    ON ca.participant_id = av.participant_id
WHERE ca.disease_type IN ('BREAST')
AND av.er_status = 'P'"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))cancer_analysis", "gel_cancer_analysis_100k") %>%
    str_replace_all("(?<=(FROM )|(JOIN ))av_tumour ", "ncras_cancer_tumour_100k ") 

dbGetQuery(connection, sql_to_run)

### Query 8: Cancer type and status - Part 1

In [ ]:
sql <- "
SELECT DISTINCT participant_id
FROM sact
WHERE analysis_group IN ('NIVOLUMAB','PEMBROLIZUMAB')
"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))sact", "ncras_systemic_anti_cancer_therapy_curated_100k") 

dbGetQuery(connection, sql_to_run)

### Query 9: Cancer type and status - Part 2

In [ ]:
sql <- "
SELECT DISTINCT sa.participant_id
FROM sact AS sa
LEFT JOIN cancer_analysis AS ca
    ON sa.participant_id = ca.participant_id
WHERE sa.analysis_group IN ('NIVOLUMAB','PEMBROLIZUMAB')
AND ca.disease_type = 'BLADDER'
"

sql_to_run <- str_replace_all(sql, "(?<=(FROM )|(JOIN ))sact", "ncras_systemic_anti_cancer_therapy_curated_100k") %>%
    str_replace_all("(?<=(FROM )|(JOIN ))cancer_analysis", "gel_cancer_analysis_100k")

dbGetQuery(connection, sql_to_run)